In [1]:
# ============================================================
# LS100 Correlation Analysis
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os
from pathlib import Path
import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 1. Paths & basic config
# ------------------------------------------------------------
BASE_DIR    = "/content/drive/MyDrive/Harvard/LS100"
DATA_DIR    = f"{BASE_DIR}/data"
HR_FILE     = f"{DATA_DIR}/LS100_Data.xlsx"
ANGLES_FILE = f"{DATA_DIR}/all_angles_master.csv"
OUT_DIR     = f"{BASE_DIR}/analysis"
os.makedirs(OUT_DIR, exist_ok=True)

# frames per second of processed videos
FPS = 60  # if you later confirm it's 30, just change this


# ------------------------------------------------------------
# 2. Helper functions
# ------------------------------------------------------------
def time_to_seconds(x):
    """Convert Time column (string or datetime.time) -> seconds."""
    if isinstance(x, str):
        try:
            return pd.to_timedelta(x).total_seconds()
        except Exception:
            return np.nan
    if hasattr(x, "hour"):  # datetime.time
        return x.hour * 3600 + x.minute * 60 + x.second
    return np.nan


def parse_video_name(v):
    """
    Expect: Aryeh_1_clip3_pose2d_angles_indices_runavg_w7
    Returns: participant, trial, clip, phase
    """
    stem = str(Path(v).stem)
    parts = stem.split("_")

    participant = parts[0]
    trial = None
    clip = None

    for p in parts[1:]:
        if p.isdigit() and trial is None:
            trial = p
        if p.startswith("clip"):
            try:
                clip = int(p.replace("clip", ""))
            except Exception:
                pass

    phase_map = {
        1: "warmup",
        2: "mile1",
        3: "mile2",
        4: "mile3",
        5: "cooldown",
    }
    phase = phase_map.get(clip, "unknown")
    return participant, trial, clip, phase


def estimate_cadence_for_clip(df_clip, fps=60):
    """
    Very rough cadence estimate from left knee angle peaks.
    Steps/min = (# peaks) / (duration_minutes)
    """
    if df_clip.empty:
        return np.nan
    # Smooth knee angles a bit
    knee = df_clip["angle_left_knee_angle"].rolling(
        window=5, center=True, min_periods=1
    ).mean()

    # Find local maxima: sign change of first derivative from + to -
    diff1 = np.diff(knee)
    sign1 = np.sign(diff1)
    diff2 = np.diff(sign1)
    # local maxima where diff2 < 0
    peaks = np.where(diff2 < 0)[0]
    n_peaks = len(peaks)

    # duration in minutes
    frames = df_clip["frame"]
    if frames.empty:
        return np.nan
    duration_sec = (frames.max() - frames.min()) / fps
    if duration_sec <= 0:
        return np.nan
    duration_min = duration_sec / 60.0

    return n_peaks / duration_min if duration_min > 0 else np.nan


# HR windows in seconds from start of trial
clip_windows = {
    1: ("warmup",   (1, 60)),
    2: ("mile1",    (4*60 + 1, 5*60)),      # 241–300
    3: ("mile2",    (13*60 + 1, 14*60)),    # 781–840
    4: ("mile3",    (23*60 + 1, 24*60)),    # 1381–1440
    5: ("cooldown", (28*60, 29*60)),        # 1680–1740
}


# ------------------------------------------------------------
# 3. Load HR data from LS100_Data.xlsx
# ------------------------------------------------------------
HR_SHEETS = {
    "Danny": "Trial 1- Danny",
    "Aryeh": "Trial 2 - Aryeh",
    "Max":   "Trial 3 - Max",
}

hr_data = {}

for person, sheet in HR_SHEETS.items():
    df = pd.read_excel(HR_FILE, sheet_name=sheet)
    df.columns = [c.strip() for c in df.columns]

    # find HR column
    hr_col = None
    for c in df.columns:
        if "heart" in c.lower():
            hr_col = c
            break
    if hr_col is None:
        raise ValueError(f"No heart-rate-like column found in {sheet}")

    # use "Time" as elapsed time column
    if "Time" not in df.columns:
        raise ValueError(f"'Time' column not found in {sheet}")
    time_col = "Time"

    clean = df[[time_col, hr_col]].copy()
    clean = clean.dropna(subset=[hr_col], how="all")
    clean["time_s"] = clean[time_col].apply(time_to_seconds)
    clean = clean.dropna(subset=["time_s"])
    clean = clean.rename(columns={hr_col: "Heart Rate"})
    clean = clean.sort_values("time_s").reset_index(drop=True)

    hr_data[person] = clean
    print(f"{person}: HR rows = {clean.shape[0]}")


def get_clip_hr(person, clip):
    """Return (mean_hr, std_hr) for given participant + clip index."""
    if person not in hr_data:
        return np.nan, np.nan
    hr_df = hr_data[person]
    win = clip_windows.get(clip)
    if win is None:
        return np.nan, np.nan
    _, (t0, t1) = win
    mask = (hr_df["time_s"] >= t0) & (hr_df["time_s"] <= t1)
    if mask.sum() == 0:
        return np.nan, np.nan
    vals = hr_df.loc[mask, "Heart Rate"]
    return vals.mean(), vals.std()


# ------------------------------------------------------------
# 4. Load angle data from all_angles_master.csv
# ------------------------------------------------------------
angles = pd.read_csv(ANGLES_FILE)
angles.columns = [c.strip() for c in angles.columns]
print("\nAngles shape:", angles.shape)
print("Angle columns:", angles.columns.tolist())

required_cols = [
    "video", "frame",
    "angle_left_knee_angle", "angle_right_knee_angle",
    "angle_left_hip_angle", "angle_right_hip_angle",
]
missing = [c for c in required_cols if c not in angles.columns]
if missing:
    raise ValueError(f"Missing angle cols: {missing}")

# parse video name
parsed = angles["video"].apply(parse_video_name)
angles[["participant", "trial", "clip", "phase"]] = pd.DataFrame(
    parsed.tolist(), index=angles.index
)

angles["time_s"] = angles["frame"] / FPS

print("\nParticipants in angles:", angles["participant"].unique())
print("Phases in angles:", angles["phase"].unique())


# ------------------------------------------------------------
# 5. Clip-level aggregation: angles
# ------------------------------------------------------------
group_cols = ["participant", "trial", "clip", "phase", "video"]
angle_cols = [
    "angle_left_knee_angle",
    "angle_right_knee_angle",
    "angle_left_hip_angle",
    "angle_right_hip_angle",
]

clip_summary = (
    angles
    .groupby(group_cols)[angle_cols]
    .agg(["mean", "std"])
    .reset_index()
)

# flatten columns
clip_summary.columns = [
    "_".join(col).strip("_") if isinstance(col, tuple) else col
    for col in clip_summary.columns
]

print("\nInitial clip_summary head:")
print(clip_summary.head())


# ------------------------------------------------------------
# 6. Add cadence, wasted energy proxy, stride placeholder
# ------------------------------------------------------------
# cadence per clip (using original angles grouped the same way)
cadence_list = []

for keys, df_clip in angles.groupby(group_cols):
    participant, trial, clip_idx, phase, video = keys
    cad = estimate_cadence_for_clip(df_clip, fps=FPS)
    cadence_list.append((*keys, cad))

cadence_df = pd.DataFrame(
    cadence_list,
    columns=group_cols + ["cadence_spm"]
)

clip_summary = clip_summary.merge(
    cadence_df,
    on=group_cols,
    how="left"
)

# wasted energy proxy: sum of knee angle stds
clip_summary["wasted_energy_proxy"] = (
    clip_summary["angle_left_knee_angle_std"] +
    clip_summary["angle_right_knee_angle_std"]
)

# stride length placeholder (requires ankle coords you don't yet have)
clip_summary["stride_length_px"] = np.nan


# ------------------------------------------------------------
# 7. Add clip-specific HR (from LS100_Data windows)
# ------------------------------------------------------------
hr_means = []
hr_stds = []

for _, row in clip_summary.iterrows():
    person = row["participant"]
    clip_idx = int(row["clip"])
    mean_hr, std_hr = get_clip_hr(person, clip_idx)
    hr_means.append(mean_hr)
    hr_stds.append(std_hr)

clip_summary["Heart Rate_mean"] = hr_means
clip_summary["Heart Rate_std"] = hr_stds

enhanced_path = f"{OUT_DIR}/clip_summary_enhanced.csv"
clip_summary.to_csv(enhanced_path, index=False)

print("\nSaved enhanced clip summary to:", enhanced_path)
print("\nEnhanced clip_summary head:")
print(clip_summary.head())


# ------------------------------------------------------------
# 8. Correlation analysis
# ------------------------------------------------------------
features = [
    "Heart Rate_mean",
    "cadence_spm",
    "wasted_energy_proxy",
    "angle_left_knee_angle_mean",
    "angle_right_knee_angle_mean",
    "angle_left_hip_angle_mean",
    "angle_right_hip_angle_mean",
]

print("\nOverall Pearson correlation (all participants, all clips):")
corr_all = clip_summary[features].corr(method="pearson")
print(corr_all)

for person in ["Danny", "Aryeh", "Max"]:
    sub = clip_summary[clip_summary["participant"] == person]
    if len(sub) < 3:
        continue
    print(f"\nPearson correlation for {person}:")
    print(sub[features].corr(method="pearson"))


# ------------------------------------------------------------
# 9. Optional plotting helpers
# ------------------------------------------------------------
import matplotlib.pyplot as plt

def scatter_hr_vs(feature, df, title_suffix=""):
    plt.figure()
    plt.scatter(df[feature], df["Heart Rate_mean"])
    plt.xlabel(feature)
    plt.ylabel("Heart Rate_mean (bpm)")
    plt.title(f"HR vs {feature} {title_suffix}")
    plt.grid(True)
    plt.show()

# Example usage:
# scatter_hr_vs("cadence_spm", clip_summary, "(all participants)")
# scatter_hr_vs("wasted_energy_proxy", clip_summary, "(all participants)")
# scatter_hr_vs("angle_left_knee_angle_mean", clip_summary, "(all participants)")


Mounted at /content/drive
Danny: HR rows = 2280
Aryeh: HR rows = 2284
Max: HR rows = 2286

Angles shape: (26880, 6)
Angle columns: ['video', 'frame', 'angle_left_knee_angle', 'angle_right_knee_angle', 'angle_left_hip_angle', 'angle_right_hip_angle']

Participants in angles: ['Aryeh' 'Danny' 'Max']
Phases in angles: ['warmup' 'mile1' 'mile2' 'mile3' 'cooldown']

Initial clip_summary head:
  participant trial  clip     phase  \
0       Aryeh     1     1    warmup   
1       Aryeh     1     2     mile1   
2       Aryeh     1     3     mile2   
3       Aryeh     2     4     mile3   
4       Aryeh     2     5  cooldown   

                                           video  angle_left_knee_angle_mean  \
0  Aryeh_1_clip1_pose2d_angles_indices_runavg_w7                  136.784933   
1  Aryeh_1_clip2_pose2d_angles_indices_runavg_w7                  131.776865   
2  Aryeh_1_clip3_pose2d_angles_indices_runavg_w7                  132.218126   
3  Aryeh_2_clip4_pose2d_angles_indices_runavg_w7      